- Take-Home exercise
    - Create a notebook which does following:
        - Insert at least **3 different coins** into the `coins` table.
        - Insert **5 daily price records** for each coin into the `prices` table.
        - Create a business question (what is the average price for each coin? what is the highest price for each coin?)
        - Write an SQL query to solve the business question

In [1]:
import pandas as pd
import sqlite3

In [3]:
conn = sqlite3.connect('crypto.db')
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS coins (
    coin_id INTEGER PRIMARY KEY,
    name TEXT,
    symbol TEXT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS prices (
    price_id INTEGER PRIMARY KEY,
    coin_id INTEGER,
    price REAL,
    date TEXT,
    FOREIGN KEY (coin_id) REFERENCES coins (coin_id)
)
""")


conn.commit()

In [4]:
coins_data = [
    (1, "Bitcoin", "BTC"),
    (2, "Ethereum", "ETH"),
    (3, "Solana", "SOL"),
    (4, "Ripple", "XRP"),
    (5, "Cardano", "ADA")
]

cursor.executemany("INSERT OR IGNORE INTO coins VALUES (?, ?, ?)", coins_data)

prices_data = [
    # Bitcoin (coin_id=1)
    (1, 1, 124000.0, "2025-08-10"),
    (2, 1, 121000.0, "2025-08-11"),
    (3, 1, 123500.0, "2025-08-12"),
    (4, 1, 124500.0, "2025-08-13"),
    (5, 1, 124223.0, "2025-08-14"),

    # Ethereum (coin_id=2)
    (6, 2, 4600.0,   "2025-08-10"),
    (7, 2, 4700.0,   "2025-08-11"),
    (8, 2, 4725.0,   "2025-08-12"),
    (9, 2, 4750.0,   "2025-08-13"),
    (10, 2, 4752.18,  "2025-08-14"),

    # Solana (coin_id=3)
    (11, 3, 180.0,    "2025-08-10"),
    (12, 3, 175.0,    "2025-08-11"),
    (13, 3, 182.0,    "2025-08-12"),
    (14, 3, 180.0,    "2025-08-13"),
    (15, 3, 180.24,   "2025-08-14"),

    # Ripple (coin_id=4)
    (16, 4, 0.32,     "2025-07-31"),
    (17, 4, 0.33,     "2025-08-01"),
    (18, 4, 0.36,     "2025-08-02"),
    (19, 4, 0.34,     "2025-08-03"),
    (20, 4, 0.33,     "2025-08-04"),

    # Cardano (coin_id=5)
    (21, 5, 0.80,     "2025-08-10"),
    (22, 5, 0.79,     "2025-08-11"),
    (23, 5, 0.80,     "2025-08-12"),
    (24, 5, 0.79,     "2025-08-13"),
    (25, 5, 0.81,     "2025-08-14"),
]

cursor.executemany("INSERT OR IGNORE INTO prices VALUES (?, ?, ?, ?)", prices_data)

conn.commit()

In [6]:
df = pd.read_sql("""
    SELECT 
        p.date,
        c.name,
        c.symbol,
        p.price
    FROM coins c
    LEFT JOIN prices p 
        ON c.coin_id = p.coin_id
""", conn)

### what is the average price for each coin?

In [9]:
average = pd.read_sql("""
    SELECT
        c.name as Coin,
        c.symbol as Symbol,
        AVG(p.price) AS "Average Price"
    FROM coins c
    LEFT JOIN prices p
        ON c.coin_id = p.coin_id
    GROUP BY c.name, c.symbol
""", conn)

print(f"Average Price for each coin:\n")
print(average)

Average Price for each coin:

       Coin Symbol  Average Price
0   Bitcoin    BTC     123444.600
1   Cardano    ADA          0.798
2  Ethereum    ETH       4705.436
3    Ripple    XRP          0.336
4    Solana    SOL        179.448


### What is the highest price for each coin

In [10]:
max_price = pd.read_sql("""
    SELECT
        c.name as Coin,
        c.symbol as Symbol,
        MAX(p.price) AS "Highest Price"
    FROM coins c
    LEFT JOIN prices p
        ON c.coin_id = p.coin_id
    GROUP BY c.name, c.symbol
""", conn)

print(f"Highest Price for each coin:\n")
print(max_price)

Highest Price for each coin:

       Coin Symbol  Highest Price
0   Bitcoin    BTC      124500.00
1   Cardano    ADA           0.81
2  Ethereum    ETH        4752.18
3    Ripple    XRP           0.36
4    Solana    SOL         182.00
